In [1]:
import pandas as pd
import numpy as np
import joblib
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from twelvedata import TDClient

# Step 1: Set up API Key (Replace with your actual API key)
API_KEY = "c4755b2a48af47e498144d182962c441"  # Updated API key
td = TDClient(apikey=API_KEY)

# Step 2: Define currency pairs
currency_pairs = ["EUR/USD", "GBP/USD", "USD/JPY", "USD/CHF", "AUD/USD", "USD/CAD"]
timeframes = ["1day"]  # Historical data
small_intervals = ["5min", "15min", "1h"]  # Live data

# Step 3: Function to fetch forex data
def fetch_forex_data(pair, interval="1day", outputsize=5000):
    try:
        data = td.time_series(
            symbol=pair,
            interval=interval,
            outputsize=outputsize,
            timezone="UTC"
        ).as_pandas()
        data["currency_pair"] = pair
        data["timeframe"] = interval
        return data
    except Exception as e:
        print(f"Error fetching {pair} ({interval}) data: {e}")
        return None

# Step 4: Fetch historical forex data
historical_data = []
for pair in currency_pairs:
    for tf in timeframes:
        print(f"Fetching historical data for {pair} ({tf})...")
        data = fetch_forex_data(pair, interval=tf, outputsize=5000)
        if data is not None:
            historical_data.append(data)
        time.sleep(10)  # Prevent API rate limits

historical_df = pd.concat(historical_data, ignore_index=False) if historical_data else None

# Step 5: Fetch live forex data for smaller timeframes
live_data = []
for pair in currency_pairs:
    for tf in small_intervals:
        print(f"Fetching live data for {pair} ({tf})...")
        live = fetch_forex_data(pair, interval=tf, outputsize=5000)
        if live is not None:
            live_data.append(live)
        time.sleep(10)

live_df = pd.concat(live_data, ignore_index=False) if live_data else None

# Step 6: Merge historical & live data
if historical_df is not None and live_df is not None:
    combined_df = pd.concat([historical_df, live_df], ignore_index=False)
elif historical_df is not None:
    combined_df = historical_df
elif live_df is not None:
    combined_df = live_df
else:
    raise ValueError("No forex data fetched!")

combined_df.reset_index(inplace=True)
print("✅ Data successfully combined. Columns:", combined_df.columns)

# Step 7: Ensure required columns exist
required_cols = ["close", "high", "low"]
for col in required_cols:
    if col not in combined_df.columns:
        raise ValueError(f"Missing '{col}' column in dataset.")

# Step 8: Calculate ATR for SL & TP
def calculate_atr(df, period=14):
    df["H-L"] = df["high"] - df["low"]
    df["H-PC"] = abs(df["high"] - df["close"].shift(1))
    df["L-PC"] = abs(df["low"] - df["close"].shift(1))
    df["TR"] = df[["H-L", "H-PC", "L-PC"]].max(axis=1)
    df["ATR"] = df["TR"].rolling(window=period).mean()
    df.drop(["H-L", "H-PC", "L-PC", "TR"], axis=1, inplace=True)
    return df

combined_df = calculate_atr(combined_df)

# Step 9: Create SL and TP based on ATR
def calculate_sl_tp(df):
    df["SL"] = np.where(df["close"].shift(-1) > df["close"],  # If next close is higher (Buy)
                        df["close"] - (df["ATR"] * 1.5),      # SL below entry
                        df["close"] + (df["ATR"] * 1.5))      # SL above entry

    df["TP"] = np.where(df["close"].shift(-1) > df["close"],
                        df["close"] + (df["ATR"] * 2),        # TP above entry for Buy
                        df["close"] - (df["ATR"] * 2))        # TP below entry for Sell
    return df

combined_df = calculate_sl_tp(combined_df)

# Step 10: Select features for ML model
features = combined_df[["close", "high", "low", "ATR"]].dropna()
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Step 11: Create labels (1 = Buy, 0 = Sell)
close_prices = combined_df["close"].values
labels = np.where(np.roll(close_prices, -1) > close_prices, 1, 0)

# Ensure labels match feature length
valid_length = features_scaled.shape[0]  # Get valid feature count
labels = labels[:valid_length]  # Trim labels to match features

# Step 12: Train-test split
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42)

# Step 13: Train the new model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 14: Save the new model
joblib.dump(model, "new_forex_model_twelvedata.pkl")

print("✅ New model trained and saved as 'new_forex_model_twelvedata.pkl'")


Fetching historical data for EUR/USD (1day)...
Fetching historical data for GBP/USD (1day)...
Fetching historical data for USD/JPY (1day)...
Fetching historical data for USD/CHF (1day)...
Fetching historical data for AUD/USD (1day)...
Fetching historical data for USD/CAD (1day)...
Fetching live data for EUR/USD (5min)...
Fetching live data for EUR/USD (15min)...
Fetching live data for EUR/USD (1h)...
Fetching live data for GBP/USD (5min)...
Fetching live data for GBP/USD (15min)...
Fetching live data for GBP/USD (1h)...
Fetching live data for USD/JPY (5min)...
Fetching live data for USD/JPY (15min)...
Fetching live data for USD/JPY (1h)...
Fetching live data for USD/CHF (5min)...
Fetching live data for USD/CHF (15min)...
Fetching live data for USD/CHF (1h)...
Fetching live data for AUD/USD (5min)...
Fetching live data for AUD/USD (15min)...
Fetching live data for AUD/USD (1h)...
Fetching live data for USD/CAD (5min)...
Fetching live data for USD/CAD (15min)...
Fetching live data for US

In [5]:
from IPython.display import FileLink
FileLink("new_forex_model_twelvedata.pkl")



C:\Users\hp\OneDrive\notebooks\new_forex_model_twelvedata.pkl